In [5]:
from idlelib.runscript import indent_message

from dotenv import load_dotenv
from sqlalchemy.sql.elements import AnnotatedColumnElement
from tensorflow.python.ops.summary_ops_v2 import graph

load_dotenv()

from typing_extensions import TypedDict
from  langchain.chat_models import init_chat_model
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

In [6]:
from typing_extensions import Annotated
from langchain.chat_models import init_chat_model

llm = init_chat_model("google_genai:gemini-2.0-flash")

class State(TypedDict):
    messages: Annotated[list, add_messages]

def chatbot(state: State) -> State:
    return {"messages": [llm.invoke(state["messages"])]}

builder = StateGraph(State)
builder.add_node("chatbot_node", chatbot)

builder.add_edge(START, "chatbot_node")
builder.add_edge("chatbot_node", END)

graph = builder.compile()


In [7]:
message = {"role": "user", "content": "Who walked on the Moon for the first time? Print only name"}
response = graph.invoke({"messages": [message]})

response["messages"]

[HumanMessage(content='Who walked on the Moon for the first time? Print only name', additional_kwargs={}, response_metadata={}, id='0c1a49f3-4feb-4678-9b13-100eb0e8033f'),
 AIMessage(content='Neil Armstrong', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='lc_run--dcaf43b6-46ff-469e-b35e-23ed218fb87d-0', usage_metadata={'input_tokens': 13, 'output_tokens': 3, 'total_tokens': 16, 'input_token_details': {'cache_read': 0}})]

In [8]:
state = None

while True:
    in_message = input("you: ")

    if in_message.lower() in {"quit", "exit"}:
        break

    if state is None:
        state = {
            "messages": [{"role": "user", "content": in_message}]
        }
    else:
        state["messages"].append({"role": "user", "content": in_message})

    # call the graph
    state = graph.invoke(state)

    # If the graph returns AIMessage objects:
    last_message = state["messages"][-1]

    # If AIMessage → use .content  (NOT indexing)
    if hasattr(last_message, "content"):
        bot_message = last_message.content
    else:
        bot_message = last_message["content"]

    print("Bot:", bot_message)


Bot: There are several people named Saksham Kumar who are in the Indian Administrative Service (IAS). To give you the most accurate information, I need a little more context. Could you provide any of the following details?

*   **Year of passing the exam/batch:** Knowing the year they joined the IAS would help narrow it down.
*   **State Cadre:** Which state are they assigned to?
*   **Any specific achievements or roles:** Do you know if they are known for anything in particular?

Without more information, it's difficult to pinpoint the exact Saksham Kumar you're looking for.


KeyboardInterrupt: Interrupted by user